# Ejercicio 4: Modelo Ising 2D

Con ChatGPT generé un código en python que evoluciona el sistema en el tiempo y usa 2 optimizaciones:
1. Calcula las probabilidades a tiempo inicial
2. Usa CPU en paralelo

In [3]:
import numpy as np
from multiprocessing import Pool
import time

L = 5#32
J = 1.0
kB = 1.0
T = 2.5
num_steps = 100#000

probabilidades = {}
for deltaE in [-8*J, -4*J, 0, 4*J, 8*J]:
    if deltaE <= 0:
        probabilidades[deltaE] = 1.0
    else:
        probabilidades[deltaE] = np.exp(-deltaE / (kB * T))

def metropolis_step_optimizado(spins):
    i, j = np.random.randint(0, L, 2)
    deltaE = 2 * J * spins[i, j] * (spins[(i+1)%L, j] + spins[(i-1)%L, j] + 
                                    spins[i, (j+1)%L] + spins[i, (j-1)%L])
    if np.random.rand() < probabilidades[deltaE]:
        spins[i, j] *= -1
    return spins

def simulacion_ising(_=None):
    spins = 2 * np.random.randint(2, size=(L, L)) - 1
    for _ in range(num_steps):
        spins = metropolis_step_optimizado(spins)
    return np.sum(spins)

def main():
    # Sin paralelización
    start_time = time.time()
    resultados_sequenciales = [simulacion_ising() for _ in range(4)]  # Ejemplo: 4 simulaciones
    end_time = time.time()
    tiempo_sequencial = end_time - start_time

    # Con paralelización
    start_time = time.time()
    with Pool() as p:
        resultados_paralelos = p.map(simulacion_ising, [() for _ in range(4)])
    end_time = time.time()
    tiempo_paralelo = end_time - start_time

    print(f"Tiempo sin paralelización: {tiempo_sequencial:.2f} segundos")
    print(f"Tiempo con paralelización: {tiempo_paralelo:.2f} segundos")

if __name__ == '__main__':
    main()
